In [1]:
import numpy as np
import neuroglancer as ng

In [2]:
# Use the BRAIN CoGS client
ng.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')

# Load in the raw data into neuroglancer and generate the link

## Host the data via cloudvolume

You will need to host your cell channel with cloudvolume: e.g.
```python
import cloudvolume
vol = cloudvolume.Cloudvolume('file:///jukebox/LightSheetData/lightserv/cz15/zimmerman_01/zimmerman_01-001/imaging_request_1/viz/rawdata/blendeddata_zimmerman_01-001')
vol.viewer(port=1337) 
```
Make sure to do this outside of this jupyter notebook in a separate python session

## Load the data into Neuroglancer and generate the link

In [22]:
viewer = ng.Viewer()
with viewer.txn() as s:
    s.layers['cell channel'] = ng.ImageLayer(source='precomputed://http://localhost:1337',
                                     shader="""void main() {emitGrayscale(1.0-toNormalized(getDataValue())*350.0);}""")
print(viewer)

http://127.0.0.1:42381/v/c48c3038430e7dd96e69788a6fbfba4fa4babc8c/


Click the link to open Neuroglancer in a new tab

If you want to load the atlas on top of this to help you find regions to annotate points, then host the raw-space atlas for this volume with cloudvolume (on port 1338, say) and load it in just like you loaded in the cell channel, but using SegmentationLayer instead of ImageLayer like: 

```python
with viewer.txn() as s:
    s.layers['raw-space atlas'] = ng.SegmentationLayer(source='precomputed://http://localhost:1338')
```

In [23]:
# Run this cell without changing anything. It creates a dictionary containing the voxel size in nm 
with viewer.txn() as s:
    dim_dict = s.dimensions
    nm_scale_dict = {}
    for coord in ['x','y','z']:
        dim_obj = dim_dict[coord]
        unit = dim_obj.unit
        assert unit == 'm'
        nm_scale_dict[coord] = dim_obj.scale*1e9
nm_scale_dict

{'x': 1800.0, 'y': 1800.0, 'z': 2000.0}

In [34]:
# Create a subvolume cutout in which to do the annotations

# Define the size in pixels, keeping in mind that 200x200x50 is a minimum
# This corresponds to ~0.35 mm x 0.35 mm x 0.1 mm
npix_subvol = [250,250,50] # x,y,z

# Decide where to put the subvolume either in mm offset from the origin (the top, front, left of the brain, 
# typically), or in voxel coordinates
# May take some experimenting
use_mm_offset = False # if you want to specify the mm offset then switch this to True and update mm_offset below.
# Otherwise update voxel-offset_origvol below to the voxel coordaintes at which you want the subvolume to start
if use_mm_offset:
    mm_offset = [4,8,2] # x y z
else:
    voxel_offset_origvol = [3200,3500,1800] # 
    mm_offset_x = voxel_offset_origvol[0]*nm_scale_dict['x']*1e-6 
    mm_offset_y = voxel_offset_origvol[1]*nm_scale_dict['y']*1e-6 
    mm_offset_z = voxel_offset_origvol[2]*nm_scale_dict['z']*1e-6 
    mm_offset = [mm_offset_x,mm_offset_y,mm_offset_z]
# mm_offset
# Don't change below this
voxel_offset = [int(round(mm_offset[0]*1e6/nm_scale_dict['x'])),
                int(round(mm_offset[1]*1e6/nm_scale_dict['y'])),
                int(round(mm_offset[2]*1e6/nm_scale_dict['z']))]
subvol = np.zeros(npix_subvol, dtype=np.uint8)

with viewer.txn() as s:
    s.layers['training_set_overlay'] = ng.ImageLayer(
        source=ng.LocalVolume(
            subvol,
            dimensions=ng.CoordinateSpace(
                scales=[nm_scale_dict['x'], nm_scale_dict['y'], nm_scale_dict['z']], 
                units=['nm', 'nm', 'nm'],
                names=['x', 'y', 'z']),
            voxel_offset=voxel_offset), 
        shader="""
void main() {
  emitGrayscale(1.0-toNormalized(getDataValue())*350.0);
}
""", 
    )
    # Then pan to the location of this new volume
    # To do that, need dimensions of your original data layer
    
    # find the position of the new subvolume in these units
    x_subvol_orig = mm_offset[0]*1e6/nm_scale_dict['x']+10
    y_subvol_orig = mm_offset[1]*1e6/nm_scale_dict['y']+10
    z_subvol_orig = mm_offset[2]*1e6/nm_scale_dict['z']+1
    # Pan/zoom to this location
#     s.voxel_coordinates = [x_subvol_orig,y_subvol_orig,z_subvol_orig]
#     s.cross_section_scale=0.5
    

You should see a square show up in at least one of the 2D panels and in the 3d viewer panel. Right click it in any of the panels to pan to it. 

In [35]:
# Make an annotation layer so you can start making annotations
with viewer.txn() as s:
    s.layers['subvol_annotation']=ng.AnnotationLayer()
    annot_layer = s.layers['subvol_annotation']
    annot_layer.tool = "annotatePoint"
    s.cross_section_depth=-40 # conrols the z-depth in microns over which the annotated points will persist. 
    # negative sign is necessary for some reason.
    # Toggle this in Neuroglancer with alt+"=" or alt+"-" to decrease and increase the depth
    s.selected_layer.layer = 'subvol_annotation' 
    s.selected_layer.visible = True
with viewer.config_state.txn() as st:
    st.status_messages['update'] = 'Start annotating points by ctrl+left clicking'   
    